In [13]:
import numpy as np
import pandas as pd
from pathlib import Path


# data_path = 'node.dat'
# nodes = pd.read_csv(data_path, header=None, sep='\t')

# # 假设文件列顺序为：ID, Name, Type, Feature
# nodes.columns = ['id', 'name', 'type', 'feature']

# # 创建一个新的列 'new_id' 用于存储每种类型从零开始的编号
# nodes['new_id'] = nodes.groupby('type').cumcount()

# # 查看结果
# print(nodes.head())

# # 保存新的node文件，如果需要的话
# nodes.to_csv('node_with_new_id.dat', sep='\t', index=False, header=False)

data_path = Path.cwd().parent.joinpath('data/bp/node.dat')

nodes = pd.read_csv(data_path, header=None, sep='\t')
nodes.columns = ['id', 'name', 'type', 'feature']
nodes['new_id'] = nodes.groupby('type').cumcount()


In [14]:
nodes.loc[nodes['type'] == 1, 'feature'] = None
# output_path = Path.cwd().parent.joinpath('dataset/bp/node.dat')
# nodes.to_csv(output_path, sep='\t', index=False, header=False)

In [16]:
import pandas as pd

# 读取节点文件，并创建原始ID到新ID的映射字典
# nodes_path = 'node_with_new_id_and_modified_feature.dat'  # 节点文件路径
# data_path = Path.cwd().parent.joinpath('data/bp/node.dat')
# nodes = pd.read_csv(data_path, header=None, sep='\t')
nodes.columns = ['id', 'name', 'type', 'feature', 'new_id']

# 创建一个字典映射，用于快速查找
id_to_new_id = nodes.set_index('id')['new_id'].to_dict()
id_to_type = nodes.set_index('id')['type'].to_dict()

# 读取link文件
link_path = Path.cwd().parent.joinpath('data/bp/link.dat')  # 请将其改为你的link文件路径
links = pd.read_csv(link_path, header=None, sep='\t')
links.columns = ['id1', 'id2', 'type', 'score']

# 定义函数用于根据类型来更新id1和id2
def update_id(row):
    type1 = id_to_type[row['id1']]
    type2 = id_to_type[row['id2']]
    if row['type'] == 0:  # 类别0的节点-类别1的节点
        if type1 == 0 and type2 == 1:
            row['id1'] = id_to_new_id[row['id1']]
            row['id2'] = id_to_new_id[row['id2']]
    elif row['type'] == 1:  # 类别0-类别0
        if type1 == 0 and type2 == 0:
            row['id1'] = id_to_new_id[row['id1']]
            row['id2'] = id_to_new_id[row['id2']]
    elif row['type'] == 2:  # 类别1-类别1
        if type1 == 1 and type2 == 1:
            row['id1'] = id_to_new_id[row['id1']]
            row['id2'] = id_to_new_id[row['id2']]
    return row

# 更新link文件中的id1和id2
links = links.apply(update_id, axis=1)

links[['id1', 'id2', 'type']] = links[['id1', 'id2', 'type']].astype(int)
# 保存更新后的link数据到新文件
output_link_path = Path.cwd().parent.joinpath('dataset/bp/updated_link.dat')  # 目标文件路径
links.to_csv(output_link_path, sep='\t', index=False, header=False)
